In [113]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns in DataFrame display
pd.set_option('display.width', None)        # Don't truncate wide rows


In [585]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel("Data/New_Final_merged_output.xlsx")

# Create the 'Property_Index' column by concatenating and uppercasing the required fields
df["Property_Index"] = (
    df["Input_Postcode"].fillna("").astype(str).str.strip() + " " +
    df["Input_PAON"].fillna("").astype(str).str.strip() + " " +
    df["Input_SAON"].fillna("").astype(str).str.strip()
).str.strip().str.upper()

# Insert 'Property_Index' as the 6th column (after the first 5 columns)
columns = df.columns.tolist()
df = df[columns[:5] + ["Property_Index"] + columns[5:]]

# Save
df.to_excel("Data/New_Final_merged_output.xlsx", index=False)

In [586]:
import pandas as pd
import numpy as np
import asyncio
from KNN_faiss import GeoKNNSearch
from property_scraper import PropertyScraper
import re  # Import the scraper class
from property_analyzer import PropertyAnalyzer

In [587]:
def load_data():
    """Loads property dataset and splits into recent (2025) and historical records."""
    data_enrich_train = pd.read_excel("Data/New_Final_merged_output.xlsx")

    return data_enrich_train

In [ ]:
def initialize_knn(data_enrich_train):
    """Initializes the KNN model with historical property data."""
    return GeoKNNSearch(
        data=data_enrich_train[['Property_Index', 'position.lat', 'position.long']],
        lat_col='position.lat', lon_col='position.long', id_col='Property_Index',
        use_exact_distance=True
    )

In [1289]:
df=load_data()

In [1290]:
new_df = df.iloc[1:].reset_index(drop=True)

In [1291]:
new_df


,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,Property_Index.1,property.type,property.tenure,property.transactionCount,property.standardPriceTransactionCount,property.additionalPriceTransactionCount,property.transaction,property.dataSource,property.standardTransactions,property.age,property.address.paon,property.address.street,property.address.town,property.address.district,property.address.county,property.address.postcode,property.address.description,property.highestPricePaid.value,property.highestPricePaid.amount,property.lowestPricePaid.value,property.lowestPricePaid.amount,property.estimatedCurrentValue.value,property.estimatedCurrentValue.amount,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,property.estimatedCurrentValue.confidence,restrictiveCovenants,epc.constructionAgeBand,epc.currentEnergyRating,epc.potentialEnergyRating,epc.glazedType,epc.propertyType,epc.builtForm,epc.localAuthority,epc.constituency,epc.floorDescription,epc.windowsDescription,epc.heatDescription,epc.hotwaterDescription,epc.wallsDescription,epc.roofDescription,epc.lightingDescription,epc.hasMainsGas,epc.numberHeatedRooms,epc.numberHabitableRooms,epc.tenure,epc.lodgementDate,epc.expiryDate,epc.mainsGasFlag,epc.energyTariff,epc.mainFuel,epc.uprn,epc.lmkKey,epc.totalFloorArea.value,epc.totalFloorArea.amount,epc.lightingCostCurrent.value,epc.lightingCostCurrent.amount,epc.lightingCostPotential.value,epc.lightingCostPotential.amount,epc.heatingCostCurrent.value,epc.heatingCostCurrent.amount,epc.heatingCostPotential.value,epc.heatingCostPotential.amount,epc.hotWaterCostCurrent.value,epc.hotWaterCostCurrent.amount,epc.hotWaterCostPotential.value,epc.hotWaterCostPotential.amount,epc.status.timestamp,epc.status.difference,epc.status.status,epc.status.message,epc.status.boolValue,epc.status.stringValue,epcRecommendations,position.x,position.y,position.lat,position.long,comparablePropertyInformation.matchingProperties,comparablePropertyInformation.lowerBandedProperties,comparablePropertyInformation.bands,comparablePropertyInformation.comparableSales,comparablePropertyInformation.currentProperty.address,comparablePropertyInformation.currentProperty.currentEnergyRating,comparablePropertyInformation.currentProperty.potentialEnergyRating,comparablePropertyInformation.currentProperty.propertyAge,comparablePropertyInformation.currentProperty.propertyType,comparablePropertyInformation.currentProperty.floorArea,comparablePropertyInformation.currentProperty.habitableRoomCount,comparablePropertyInformation.currentProperty.heatedRoomCount,comparablePropertyInformation.currentProperty.councilTax.band,comparablePropertyInformation.currentProperty.councilTax.price,comparablePropertyInformation.currentProperty.councilTax.address,property.yearBuilt,property.address.locality,leases,property.additionalTransactions,comparablePropertyInformation.comparableProperties,property.address.saon,code,message,details
0,c95444463147530ca2ace31548b14658,57,NaN,st johns avenue,sw15 6al,SW15 6AL 57,SW15 6AL 57,Detached,Freehold,2.0,2.0,0.0,"[{'date': {'full': '18 July 2018', 'short': '1...",internal,"[{'date': {'full': '18 July 2018', 'short': '1...",2007 onwards,57,St Johns Avenue,London,Wandsworth,Greater London,SW15 6AL,"57, St. John's Avenue",2765000,"£2,765,000",1700000,"£1,700,000",3104865,"£3,104,865",287,10818.344948,High,"[{'address': '57 St Johns Avenue, London Sw15 ...",2007 onwards,B,B,Double Glazing Installed During Or After 2002,House,Detached,Wandsworth,Putney,"Solid, Insulated (Assumed)",Fully Double Glazed,"Boiler And Underfloor Heating, Mains Gas",From Main System,"Cavity Wall, As Built, Insulated (Assumed)","Pitched, Insulated (Assumed)",Low Energy Lighting In All Fixed Outlets,1,10,10,Owner Occupied,30 January 2018,30 January 2028,Y,Unknown,mains gas (not community),121002612,32136039762018013018354823908228,287,287m²,116,£116,116,£116,766,£766,766,£766,121,£121,121,£121,2028-01-30T00:00:00+00:00,983,Pass

In [1292]:
columns_to_keep = [
    "Matched_PID",
    "Input_PAON",
    "Input_SAON",
    "Input_Street",
    "Input_Postcode",
    "Property_Index",
    "property.type",
    "property.tenure",
    "property.age",
    "position.lat",
    "position.long",
    "epc.numberHeatedRooms",
    "epc.numberHabitableRooms",
    "property.estimatedCurrentValue.floorArea",
    "property.estimatedCurrentValue.pricePerSqm"
]
new_df = new_df[columns_to_keep]


In [1293]:
new_df.columns

Index(['Matched_PID', 'Input_PAON', 'Input_SAON', 'Input_Street',
       'Input_Postcode', 'Property_Index', 'property.type', 'property.tenure',
       'property.age', 'position.lat', 'position.long',
       'epc.numberHeatedRooms', 'epc.numberHabitableRooms',
       'property.estimatedCurrentValue.floorArea',
       'property.estimatedCurrentValue.pricePerSqm'],
      dtype='object')

In [1295]:
new_df['position.long'] = pd.to_numeric(new_df['position.long'], errors='coerce')

C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_18556\4267752092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['position.long'] = pd.to_numeric(new_df['position.long'], errors='coerce')


In [1330]:
new_df.columns

Index(['Matched_PID', 'Input_PAON', 'Input_SAON', 'Input_Street',
       'Input_Postcode', 'Property_Index', 'property.type', 'property.tenure',
       'property.age', 'position.lat', 'position.long',
       'epc.numberHeatedRooms', 'epc.numberHabitableRooms',
       'property.estimatedCurrentValue.floorArea',
       'property.estimatedCurrentValue.pricePerSqm'],
      dtype='object')

In [1350]:
new_df.loc[new_df['Property_Index'] == 'N20 8LR 27', 'property.estimatedCurrentValue.pricePerSqm'] = 10656.26

In [1354]:
new_df[new_df['Input_Postcode']=='en5 3ll']

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm
24445,398b462f8d3fd9df588fb622402b2b38,troika 126,NaN,barnet road,en5 3ll,EN5 3LL TROIKA 126,Detached,Freehold,2018,51.647341,-0.234875,14,13,865.16,6363.66


In [1352]:
# Step 1: Print duplicated values in 'Matched_PID'
duplicates = new_df['Matched_PID'][new_df['Matched_PID'].duplicated()]
print("Duplicated Matched_PID values:")
print(duplicates.unique())  # Just the unique ones that appear more than once

# Step 2: Drop duplicate rows based on 'Matched_PID', keeping the first occurrence
new_df = new_df.drop_duplicates(subset='Matched_PID', keep='first')



Duplicated Matched_PID values:
['bda56eea7da435c6f100672cc65c8a37' 'df3f6799e0d96197a86286ace3897166'
 'f17047f288d87cb70f91c9187e39a4dc' ... '8b7aba5ffda6c76d7268dbc57b937516'
 '1764e59e307357f49e2ac13ddb8d9393' 'a2bba871759c4d241b0d9878e071565d']


In [1353]:
# Replace specific values
new_df['property.type'] = new_df['property.type'].replace({
    'Semi-Detached': 'Semi Detached',
    'No Data!': 'No Data'
})

# Drop rows where property.type is NaN
new_df = new_df.dropna(subset=['property.type'])
len(new_df)


23047

In [956]:
new_df.tail(24)

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm
24425,fbd9f1f40a3149e2b884bd9899e8b9eb,2,NaN,frognal gardens,nw3 6ux,NW3 6UX 2,Detached,Freehold,1930-1949,51.555668,51.555668,3,3,525,19908.571429
24426,88e3c439a0769cd4d4359d1d4f1c2fa2,14,NaN,lansdowne crescent,w11 2nj,W11 2NJ 14,Detached,Freehold,before 1900,51.511933,51.511933,19,19,582,41443.298969
24427,46d2046f5e08057041bcc2c50bb4695d,4,NaN,makepeace avenue,n6 6ej,N6 6EJ 4,Detached,Freehold,1900-1929,51.564093,51.564093,8,8,168,16151.785714
24428,b37dc28ce68be087b1ea04f4480aa0f7,2,NaN,bettridge road,sw6 3qd,SW6 3QD 2,End Terrace,Freehold,1900-1929,51.471016,51.471016,5,5,162,13745.537037
24430,d9fa61a6e97215a80b2d0774e954b817,6,NaN,hungershall park,tn4 8nd,TN4 8ND 6,Detached,Freehold,before 1900,51.125895,51.125895,14,14,784,4430.996173
24431,7f177e316845c0e95c446f6a82989bae,50,NaN,hereford road,w2 5aj,W2 5AJ 50,Mid Terrace,Freehold,1900-1929,51.516127,51.516127,10,10,261,21415.528736
24432,9d6bea1f32b8d75b27b9c75e7711dbe6,21,NaN,broadlands road,n6 4ae,N6 4AE 21,Detached,Freehold,before 1900,51.574971,51.574971,15,15,543,13649.060773
24433,155f3a398deb588da72af742a86678a5,5,NaN,roedean crescent,sw15 5jx,SW15 5JX 5,Detached,Freehold,1930-1949,51.457883,51.457883,No Data,No Data,1003,5520.87338
24435,af95f0e4855b9c4d5a63038e94e13460,40,NaN,norland square,w11 4pz,W11 4PZ 40,Mid Terrace,Freehold,before 1900,51.507029,51.507029,10,10,307,39962.7557
24437,fc803044b7991207fa52b0aac82b3b76,wyndham house,flat 2,sloane square,sw1w 8ar,SW1W 8AR WYNDHAM HOUSE FLAT 2,End Terrace,Leasehold,before 1900,51.492019,51.492019,5,5,No Data,No Data


In [1355]:
sample = new_df[new_df['Property_Index'] == 'EN5 3LL TROIKA 126'].iloc[0]
sample

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                         Detached
property.tenure                                                       Freehold
property.age                                                              2018
position.lat                                                         51.647341
position.long                                                        -0.234875
epc.numberHeatedRooms                                                       14
epc.numberHabitableRooms                            

In [1356]:
new_df['property.type'].value_counts()

property.type
Detached                11118
Mid Terrace              3858
Flat/Maisonette          3463
Semi Detached            2238
End Terrace              1541
Terraced                  325
Enclosed End Terrace      249
Enclosed Mid Terrace      200
Other                      35
No Data                    20
Name: count, dtype: int64

In [1064]:
sample['property.type']='Flat/Maisonette'

In [1066]:
sample['property.age']=2015
sample['position.lat']=51.5209001
sample['position.long']=-0.1602064
sample['epc.numberHeatedRooms']=7
sample['epc.numberHabitableRooms']=2
sample['property.estimatedCurrentValue.floorArea']=118.07
sample['property.estimatedCurrentValue.pricePerSqm']=22649.92

In [1067]:
sample

Matched_PID                                   30f74dcf21950f09fece8beffefabf1f
Input_PAON                                                                  24
Input_SAON                                                                 NaN
Input_Street                                                 paddington street
Input_Postcode                                                         w1u 5qy
Property_Index                                                      W1U 5QY 24
property.type                                                  Flat/Maisonette
property.tenure                                                       Freehold
property.age                                                              2015
position.lat                                                           51.5209
position.long                                                        -0.160206
epc.numberHeatedRooms                                                        7
epc.numberHabitableRooms                            

In [ ]:
# ###Troike 126 
# sample['property.age']='2018'
# sample['epc.numberHeatedRooms']=20
# sample['epc.numberHabitableRooms']=5
# sample['position.lat']=51.6476017
# sample['position.long']=-0.2311487
# sample['property.estimatedCurrentValue.floorArea']=864.55
# sample['property.estimatedCurrentValue.pricePerSqm']=6360.53
# sample

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                         Detached
property.tenure                                                       Freehold
property.age                                                              2018
position.lat                                                         51.647602
position.long                                                        -0.231149
epc.numberHeatedRooms                                                       20
epc.numberHabitableRooms                            

In [ ]:
# sample['epc.numberHeatedRooms']=12
# sample['epc.numberHabitableRooms']=6
# sample['property.estimatedCurrentValue.floorArea']=707.02
# sample['property.estimatedCurrentValue.pricePerSqm']=6250.02

C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_18556\2282255659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['epc.numberHeatedRooms']=12
C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_18556\2282255659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['epc.numberHabitableRooms']=6
C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_18556\2282255659.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample[

In [1357]:
latitude=[sample["position.lat"]]
longitude=[sample["position.long"]]
latitude_np = np.float64(latitude[0])
longitude_np=np.float64(longitude[0])
sample

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                         Detached
property.tenure                                                       Freehold
property.age                                                              2018
position.lat                                                         51.647341
position.long                                                        -0.234875
epc.numberHeatedRooms                                                       14
epc.numberHabitableRooms                            

In [ ]:
# ### 24 Padddington street
# sample['position.lat']=51.520569
# sample['position.long']=-0.155164
# sample['epc.numberHeatedRooms']=4
# sample['epc.numberHabitableRooms']=2
# sample['property.estimatedCurrentValue.floorArea']=319.952
# sample['property.estimatedCurrentValue.pricePerSqm']=22652.31

In [158]:
# sample['epc.numberHabitableRooms']=5
# sample['epc.numberHeatedRooms']=11
# sample['property.estimatedCurrentValue.floorArea']=9306
# sample['property.estimatedCurrentValue.pricePerSqm']=6362.69
# sample['property.age']=2018

In [159]:
# sample['property.estimatedCurrentValue.floorArea']=351
# sample['property.estimatedCurrentValue.pricePerSqm']=21356.83

In [160]:
# ### 12 Carlton Hill
# sample['property.age']=1850
# sample['epc.numberHeatedRooms']=14
# sample['epc.numberHabitableRooms']=14
# sample['property.estimatedCurrentValue.floorArea']=335
# sample['property.estimatedCurrentValue.pricePerSqm']=18648.44

In [161]:
# sample['property.age']=1890
# sample['epc.numberHeatedRooms']=14
# sample['epc.numberHabitableRooms']=14
# sample['property.estimatedCurrentValue.floorArea']=390    
# sample['property.estimatedCurrentValue.pricePerSqm']=20612.45


In [1358]:
import pandas as pd

# Define invalid values
invalid_values = ['', '0', None, float('nan'), 'No Data']

# Replace invalid values with NA for consistency
new_df['position.lat'] = new_df['position.lat'].replace(invalid_values, pd.NA)
new_df['position.long'] = new_df['position.long'].replace(invalid_values, pd.NA)

# Identify rows to remove
mask = new_df['position.lat'].isna() | new_df['position.long'].isna()

# Extract removed rows
removed_rows = new_df[mask]

# Print Property_Index of removed rows
print("Removed Property_Index values:")
print(removed_rows['Property_Index'])

# Print number of removed rows
print(f"\nTotal rows removed: {len(removed_rows)}")

# Remove the invalid rows from the original DataFrame
new_df = new_df[~mask].reset_index(drop=True)

Removed Property_Index values:
39                                   SW18 3LH 16
41                       TN17 1JJ LIDWELLS HOUSE
46                      SW19 5DG THURLOE HOUSE 9
49                           BH19 3LQ WESTMANTON
56                            BN16 1SG LUNA VIEW
                          ...                   
24414    BS20 7HY BEACHCLIFF BEACHCLIFF HOUSE 1A
24418                           W11 3LX 5 FLAT 5
24419                         SW7 5PN 41 FLAT 14
24420                                 SW3 2HH 90
24451                                 W1U 5QY 24
Name: Property_Index, Length: 3927, dtype: object

Total rows removed: 3927


In [644]:
new_df.loc[new_df['Property_Index'] == 'SW1X 9BS ROYAL COURT HOUSE FLAT 34', 'property.estimatedCurrentValue.pricePerSqm'] = 20809.71

In [962]:
new_df[new_df['Input_Postcode']=='en4 0jp']

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm
9712,a9b523204741cf2c00c93b360dd6e807,30,NaN,beech hill,en4 0jp,EN4 0JP 30,Detached,Freehold,NaN,51.666735,51.666735,NaN,NaN,320,19332.878125
10543,a4640f9a8d3ba42a54e0a5c7c7e89cd1,sereno house 22a,flat 9,beech hill,en4 0jp,EN4 0JP SERENO HOUSE 22A FLAT 9,End Terrace,Leasehold,2018,51.666851,51.666851,NaN,NaN,225,12612.804444
15066,245172725c58b0d5ef303253a013d1de,sereno house 22,flat 1,beech hill,en4 0jp,EN4 0JP SERENO HOUSE 22 FLAT 1,Flat/Maisonette,Leasehold,2018,51.666851,51.666851,NaN,NaN,176,11949.857955
18026,3f6e16162f04dacb0d0d3d8ae8713fed,10,NaN,beech hill,en4 0jp,EN4 0JP 10,Detached,Freehold,1900-1929,51.66708,51.66708,10,10,270,9139.27037
18641,84f2bfa2526ea82af87fe62933a27cbc,28,NaN,beech hill,en4 0jp,EN4 0JP 28,Detached,Freehold,1996-2002,51.663502,-0.175593,11,11,590,7590.771186
18874,d2a6aeebe0242aa61f94608d34c3da7e,8b,NaN,beech hill,en4 0jp,EN4 0JP 8B,Detached,Freehold,1996-2002,51.6670897,-0.1617361,19,7,762,7398.248031


In [1359]:
subject_property_coordinates = (latitude_np, longitude_np)
knn = initialize_knn(new_df)
property_indices, distances = knn.knearest(subject_property_coordinates, 1000, return_distances=True)
# Convert to a dictionary to get unique indices and their minimum distances
potential_comparables = pd.DataFrame({
    'Property_Index': property_indices,
    'Physical Distance': distances
})

# Step 3: Merge with full property data
pc_df = new_df.merge(potential_comparables, on='Property_Index', how='inner')

# Step 4: Deduplicate: keep closest property per index (or customize your logic)
pc_df = pc_df.sort_values(by='Physical Distance').drop_duplicates(subset='Property_Index', keep='first')

In [967]:
pc_df[pc_df['Input_Postcode']=='en4 0nl']

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance
980,588f3470ac40fc72f6616ff96f7bab70,109,NaN,camlet way,en4 0nl,EN4 0NL 109,Detached,Freehold,1900-1929,51.663589,-0.189668,12,12,394,10306.817259,3.368788


In [1360]:
import re


def clean_property_index(value):
    if isinstance(value, str):
        value = value.lower()  # Optional: makes matching case-insensitive
        value = re.sub(r"[,’']", '', value)  # Remove both straight and curly apostrophes
        value = re.sub(r'[,\-\.]', '', value)
        value = re.sub(r'\s+', ' ', value)   # Replace multiple spaces with one
        return value.strip()
    return value


# Step 1: Load data
new_potential_comparables = pd.read_excel("Data/premium_property_enrich_individual_properties.xlsx")

# Step 2: Ensure 'Property_Index' exists
if 'Property_Index' not in new_potential_comparables.columns:
    raise ValueError("The new dataframe does not have a 'Property_Index' column.")

# Step 3: Clean Property_Index in both DataFrames
new_potential_comparables['Cleaned_Property_Index'] = new_potential_comparables['Property_Index'].apply(clean_property_index)
pc_df['Cleaned_Property_Index'] = pc_df['Property_Index'].apply(clean_property_index)

# Step 4: Group by cleaned index and sum price
price_df = new_potential_comparables.groupby('Cleaned_Property_Index', as_index=False)['Price'].sum()

# Step 5: Merge on cleaned index
pc_with_price = pc_df.merge(price_df, on='Cleaned_Property_Index', how='left')

# Step 6: Identify unmatched rows
unmatched = pc_with_price[pc_with_price['Price'].isna()]
if not unmatched.empty:
    print("Unmatched Property_Index rows (original):")
    print(unmatched['Property_Index'].tolist())
    print(f"Total unmatched rows: {len(unmatched)}")

# Step 7: Drop unmatched


Unmatched Property_Index rows (original):
['EN5 3LL TROIKA 126']
Total unmatched rows: 1


In [ ]:
# pc_with_price = pc_with_price.dropna(subset=['Price']).reset_index(drop=True)

In [1247]:
pc_with_price[pc_with_price['Input_Postcode']=='hp9 2ln']


,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area
1,e4d0e51ef4bb3484937c4773b3a4f48a,eaton house 45,NaN,penn road,hp9 2ln,HP9 2LN EATON HOUSE 45,Detached,Freehold,2019,51.616265,-0.651105,13,10,836.84,8124.85,0.118977,hp9 2ln eaton house 45,7200000.0,8603.79523


In [ ]:
# pc_with_price.loc[pc_with_price['Property_Index'] == 'SW3 6DZ 3', 'property.tenure']='Freehold'


In [1361]:
import numpy as np

mask = (
    pc_with_price['property.estimatedCurrentValue.pricePerSqm'].isna() |
    (pc_with_price['property.estimatedCurrentValue.pricePerSqm'] == '') |
    (pc_with_price['property.estimatedCurrentValue.pricePerSqm'] == 'No Data')
)

pc_with_price.loc[mask, 'property.estimatedCurrentValue.pricePerSqm'] = (
    pc_with_price.loc[mask, 'Price'] /
    pc_with_price.loc[mask, 'property.estimatedCurrentValue.floorArea']
)


In [1362]:
pc_with_price['Sold Price Per Area']=pc_with_price['Price']/pc_with_price['property.estimatedCurrentValue.floorArea']


In [692]:
# import pandas as pd
# import json
# import re

# # Load the Excel file
# df = pd.read_excel("Data/Final_merged_output1.xlsx")

# # Get the last row
# row = df.iloc[-1]

# # Get the value from 'property.transaction' column
# transactions_str = row["property.transaction"]

# if pd.notna(transactions_str) and transactions_str.strip():
#     try:
#         # Convert single quotes to double quotes
#         json_compatible_str = transactions_str.replace("'", '"')

#         # Replace True/False with true/false
#         json_compatible_str = re.sub(r'\bTrue\b', 'true', json_compatible_str)
#         json_compatible_str = re.sub(r'\bFalse\b', 'false', json_compatible_str)

#         # Parse it as JSON
#         transactions = json.loads(json_compatible_str)

#         # Find the most recent transaction by 'iso' date
#         most_recent = max(transactions, key=lambda x: x['date']['iso'])

#         # Extract the price paid
#         price_paid = most_recent['pricePaid']['value']

#         print(f"Most recent pricePaid value: {price_paid}")

#     except Exception as e:
#         print(f"Error parsing transaction data: {e}")
# else:
#     print("No valid transaction data in the last row.")


In [1211]:
sample

Matched_PID                                   2829a77a311aa3a7f0ae4fa58aa8ebba
Input_PAON                                                                  56
Input_SAON                                                                 NaN
Input_Street                                                         penn road
Input_Postcode                                                         hp9 2ls
Property_Index                                                      HP9 2LS 56
property.type                                                         Detached
property.tenure                                                       Freehold
property.age                                                         1930-1949
position.lat                                                         51.617324
position.long                                                        -0.650871
epc.numberHeatedRooms                                                       14
epc.numberHabitableRooms                            

In [1363]:
sample['Price']=5500000
sample['Sold Price Per Area']=sample['Price']/sample['property.estimatedCurrentValue.floorArea']
sample

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                         Detached
property.tenure                                                       Freehold
property.age                                                              2018
position.lat                                                         51.647341
position.long                                                        -0.234875
epc.numberHeatedRooms                                                       14
epc.numberHabitableRooms                            

In [751]:
# import pandas as pd

# # Step 1: Load the Excel file
# df = pd.read_excel("Data/premium_property_enrich_individual_properties.xlsx")

# # Step 2: Get the Property_Index from the sample Series
# property_index = sample['Property_Index']

# # Step 3: Filter the dataframe to match the Property_Index
# matching_rows = df[df['Property_Index'] == property_index]

# if not matching_rows.empty:
#     # Step 4: Sum the Price values if there are duplicates
#     total_price = matching_rows['Price'].sum()

#     # Step 5: Add Price to the sample Series
#     sample['Price'] = total_price
# else:
#     print(f"No match found for Property_Index: {property_index}")


In [752]:
# # Step 6: Find additional rows matching Street, excluding same Property_Index and already-included ones
# additional_rows = pc_with_price[
#     (pc_with_price['Input_Street'] == sample['Input_Street']) &
#     (pc_with_price['Property_Index'] != sample['Property_Index']) &
#     (~pc_with_price['Property_Index'].isin(final_potential_comparables['Property_Index']))
# ]

# # Step 7: Add those rows to final_potential_comparables
# final_potential_comparables = pd.concat([final_potential_comparables, additional_rows], ignore_index=True)


In [1364]:
import pandas as pd
import numpy as np

# Define columns to check
columns_to_check = [
    "property.type",
    "property.tenure",
    "property.age",
    "position.lat",
    "position.long",
    "epc.numberHeatedRooms",
    "epc.numberHabitableRooms",
    "property.estimatedCurrentValue.floorArea",
    "property.estimatedCurrentValue.pricePerSqm",
    "Price",
    "Sold Price Per Area"
]

# Step 1: Replace blank strings with NaN
df_cleaned = pc_with_price.copy()
df_cleaned[columns_to_check] = df_cleaned[columns_to_check].replace(r'^\s*$', np.nan, regex=True)

# Step 2: Count rows before dropping
initial_rows = df_cleaned.shape[0]

# Step 3: Drop rows with any NaN in the selected columns
df_cleaned = df_cleaned.dropna(subset=columns_to_check).reset_index(drop=True)

# Step 4: Count rows after dropping
final_rows = df_cleaned.shape[0]
dropped_rows = initial_rows - final_rows

print(f"Removed {dropped_rows} rows due to missing or invalid values in critical columns.")

# Optional: Assign back to your original variable
final_potential_comparables = df_cleaned


Removed 175 rows due to missing or invalid values in critical columns.


C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_18556\938468442.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_cleaned[columns_to_check] = df_cleaned[columns_to_check].replace(r'^\s*$', np.nan, regex=True)


In [1365]:
final_potential_comparables

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area
0,5e431fc077ca272c2eb8b0f1d601b606,rowley ridge 20,NaN,rowley green road,en5 3hj,EN5 3HJ ROWLEY RIDGE 20,Detached,Freehold,2024,51.647251,-0.236892,16.0,17.0,1013.95,5920.150000,0.139507,en5 3hj rowley ridge 20,6000000.0,5917.451551
1,f3a87fa3ad3efb49a5d1b418875a5325,27,NaN,totteridge common,n20 8lr,N20 8LR 27,Detached,Freehold,1950-1966,51.632384,-0.218331,10.0,10.0,483.00,10656.260000,2.017231,n20 8lr 27,6250000.0,12939.958592
2,588f3470ac40fc72f6616ff96f7bab70,109,NaN,camlet way,en4 0nl,EN4 0NL 109,Detached,Freehold,1900-1929,51.663589,-0.189668,12.0,12.0,452.71,10306.817259,3.604147,en4 0nl 109,4250000.0,9387.908374
3,84f2bfa2526ea82af87fe62933a27cbc,28,NaN,beech hill,en4 0jp,EN4 0JP 28,Detached,Freehold,1996-2002,51.663502,-0.175593,11.0,11.0,590.00,7590.771186,4.466931,en4 0jp 28,4650000.0,7881.355932
4,d2a6aeebe0242aa61f94608d34c3da7e,8b,NaN,beech hill,en4 0jp,EN4 0JP 8B,Detached,Freehold,2010,51.667090,-0.161736,17.0,12.0,762.00,7398.248031,5.502485,en4 0jp 8b,5900000.0,7742.782152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,2f63e82290634c1bd33153c14c3aaf6a,114,NaN,runnymede road,ne20 9hl,NE20 9HL 114,Detached,Freehold,2003-2006,55.042508,55.042508,11.0,11.0,921.00,8136.640608,3592.006252,ne20 9hl 114,5800000.0,6297.502714
821,eeae48a0d21a6666fd9b33103b56d518,64,NaN,darras road,ne20 9pg,NE20 9PG 64,Detached,Freehold,2003-2006,55.043492,55.043492,8.0,8.0,514.00,5579.410506,3592.033399,ne20 9pg 64,2450000.0,4766.536965
822,230a0f5cad4035d05b7d78fd3ec6eed3,66,NaN,runnymede road,ne20 9hh,NE20 9HH 66,Detached,Freehold,2007-2011,55.045479,55.045479,11.0,11.0,1564.00,4330.549233,3592.088219,ne20 9hh 66,11000000.0,7033.248082
823,f1bafca1e8c1710494aab961122311eb,bolbec chase,NaN,whalton park,ne61 3tu,NE61 3TU BOLBEC CHASE,Detached,Freehold,1996-2002,55.133641,55.133641,21.0,21.0,937.00,4243.790822,3594.522272,ne61 3tu bolbec chase,4000000.0,4268.943436


In [1366]:
sample['Physical Distance']=0.0

In [1367]:
sample

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                         Detached
property.tenure                                                       Freehold
property.age                                                              2018
position.lat                                                         51.647341
position.long                                                        -0.234875
epc.numberHeatedRooms                                                       14
epc.numberHabitableRooms                            

In [1368]:
import pandas as pd
import numpy as np

# Columns
categorical_cols = ['property.type', 'property.tenure', 'property.age']
amount_cols = [
    'epc.numberHeatedRooms', 'epc.numberHabitableRooms','property.estimatedCurrentValue.floorArea','property.estimatedCurrentValue.pricePerSqm'
]

# Make a copy
df = final_potential_comparables.copy()

# === Step 1: Clean and convert amount columns ===
def clean_currency(val):
    if isinstance(val, str):
        val = val.replace("£", "").replace(",", "").strip()
    try:
        return float(val)
    except:
        return np.nan

for col in categorical_cols:
    df[col] = df[col].astype(str)

for col in amount_cols:
    df[col] = df[col].apply(clean_currency)

# === Step 2: Encode categorical columns ===
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df[categorical_cols] = encoder.fit_transform(df[categorical_cols])

# === Step 3: Apply same transformation to sample (Series) ===
def preprocess_sample(sample, encoder, categorical_cols, amount_cols):
    sample = sample.copy()

    # Clean amounts
    for col in amount_cols:
        val = sample[col]
        if isinstance(val, str):
            val = val.replace("£", "").replace(",", "").strip()
        try:
            sample[col] = float(val)
        except:
            sample[col] = np.nan

    # Encode categoricals
    cat_values = sample[categorical_cols].values.reshape(1, -1)
    sample[categorical_cols] = encoder.transform(cat_values)[0]

    return sample

# Apply to sample
sample = preprocess_sample(sample, encoder, categorical_cols, amount_cols)


c:\Users\ArjunShankar\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(


In [1369]:
final_df=df.copy()

In [1370]:
final_df.columns

Index(['Matched_PID', 'Input_PAON', 'Input_SAON', 'Input_Street',
       'Input_Postcode', 'Property_Index', 'property.type', 'property.tenure',
       'property.age', 'position.lat', 'position.long',
       'epc.numberHeatedRooms', 'epc.numberHabitableRooms',
       'property.estimatedCurrentValue.floorArea',
       'property.estimatedCurrentValue.pricePerSqm', 'Physical Distance',
       'Cleaned_Property_Index', 'Price', 'Sold Price Per Area'],
      dtype='object')

In [1371]:
final_df

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area
0,5e431fc077ca272c2eb8b0f1d601b606,rowley ridge 20,NaN,rowley green road,en5 3hj,EN5 3HJ ROWLEY RIDGE 20,0.0,0.0,23.0,51.647251,-0.236892,16.0,17.0,1013.95,5920.150000,0.139507,en5 3hj rowley ridge 20,6000000.0,5917.451551
1,f3a87fa3ad3efb49a5d1b418875a5325,27,NaN,totteridge common,n20 8lr,N20 8LR 27,0.0,0.0,2.0,51.632384,-0.218331,10.0,10.0,483.00,10656.260000,2.017231,n20 8lr 27,6250000.0,12939.958592
2,588f3470ac40fc72f6616ff96f7bab70,109,NaN,camlet way,en4 0nl,EN4 0NL 109,0.0,0.0,0.0,51.663589,-0.189668,12.0,12.0,452.71,10306.817259,3.604147,en4 0nl 109,4250000.0,9387.908374
3,84f2bfa2526ea82af87fe62933a27cbc,28,NaN,beech hill,en4 0jp,EN4 0JP 28,0.0,0.0,8.0,51.663502,-0.175593,11.0,11.0,590.00,7590.771186,4.466931,en4 0jp 28,4650000.0,7881.355932
4,d2a6aeebe0242aa61f94608d34c3da7e,8b,NaN,beech hill,en4 0jp,EN4 0JP 8B,0.0,0.0,19.0,51.667090,-0.161736,17.0,12.0,762.00,7398.248031,5.502485,en4 0jp 8b,5900000.0,7742.782152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,2f63e82290634c1bd33153c14c3aaf6a,114,NaN,runnymede road,ne20 9hl,NE20 9HL 114,0.0,0.0,13.0,55.042508,55.042508,11.0,11.0,921.00,8136.640608,3592.006252,ne20 9hl 114,5800000.0,6297.502714
821,eeae48a0d21a6666fd9b33103b56d518,64,NaN,darras road,ne20 9pg,NE20 9PG 64,0.0,0.0,13.0,55.043492,55.043492,8.0,8.0,514.00,5579.410506,3592.033399,ne20 9pg 64,2450000.0,4766.536965
822,230a0f5cad4035d05b7d78fd3ec6eed3,66,NaN,runnymede road,ne20 9hh,NE20 9HH 66,0.0,0.0,17.0,55.045479,55.045479,11.0,11.0,1564.00,4330.549233,3592.088219,ne20 9hh 66,11000000.0,7033.248082
823,f1bafca1e8c1710494aab961122311eb,bolbec chase,NaN,whalton park,ne61 3tu,NE61 3TU BOLBEC CHASE,0.0,0.0,8.0,55.133641,55.133641,21.0,21.0,937.00,4243.790822,3594.522272,ne61 3tu bolbec chase,4000000.0,4268.943436


In [1372]:
columns_to_drop = [
    'Matched_PID',
    'Input_PAON',
    'Input_Postcode',
    'Input_SAON',
    'Input_Street',
    'property.tenure',
    'Property_Index',
    'position.lat',
    'position.long',
    'Price',
    'Cleaned_Property_Index',
    'Sold Price Per Area'
]

final_df = final_df.drop(columns=columns_to_drop)

In [1373]:
final_df

,property.type,property.age,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance
0,0.0,23.0,16.0,17.0,1013.95,5920.150000,0.139507
1,0.0,2.0,10.0,10.0,483.00,10656.260000,2.017231
2,0.0,0.0,12.0,12.0,452.71,10306.817259,3.604147
3,0.0,8.0,11.0,11.0,590.00,7590.771186,4.466931
4,0.0,19.0,17.0,12.0,762.00,7398.248031,5.502485
...,...,...,...,...,...,...,...
820,0.0,13.0,11.0,11.0,921.00,8136.640608,3592.006252
821,0.0,13.0,8.0,8.0,514.00,5579.410506,3592.033399
822,0.0,17.0,11.0,11.0,1564.00,4330.549233,3592.088219
823,0.0,8.0,21.0,21.0,937.00,4243.790822,3594.522272


In [1374]:
sample1=sample.copy()
sample1

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                              0.0
property.tenure                                                            0.0
property.age                                                              -1.0
position.lat                                                         51.647341
position.long                                                        -0.234875
epc.numberHeatedRooms                                                     14.0
epc.numberHabitableRooms                            

In [1375]:
sample1

Matched_PID                                   398b462f8d3fd9df588fb622402b2b38
Input_PAON                                                          troika 126
Input_SAON                                                                 NaN
Input_Street                                                       barnet road
Input_Postcode                                                         en5 3ll
Property_Index                                              EN5 3LL TROIKA 126
property.type                                                              0.0
property.tenure                                                            0.0
property.age                                                              -1.0
position.lat                                                         51.647341
position.long                                                        -0.234875
epc.numberHeatedRooms                                                     14.0
epc.numberHabitableRooms                            

In [1376]:
columns_to_drop = [
    'Matched_PID',
    'Input_PAON',
    'Input_Postcode',
    'Input_SAON',
    'Input_Street',
    'Property_Index',
    'property.tenure',
    'position.lat',
    'position.long',
    'Price',
    'Sold Price Per Area'
]

sample1 = sample1.drop(labels=columns_to_drop, errors='ignore')

In [1377]:
columns_to_scale = [
    'property.age',
    'epc.numberHeatedRooms',
    'epc.numberHabitableRooms',
    'property.estimatedCurrentValue.floorArea',
    'property.estimatedCurrentValue.pricePerSqm'
]


In [1378]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
final_df[columns_to_scale] = scaler.fit_transform(final_df[columns_to_scale])


In [1379]:
# Ensure columns in sample are in same order
sample_scaled = sample1.copy()

# Convert to DataFrame and transpose to a row
sample_df = sample_scaled[columns_to_scale].to_frame().T

# Apply same scaler
scaled_values = scaler.transform(sample_df)

# Assign back to sample
for i, col in enumerate(columns_to_scale):
    sample_scaled[col] = scaled_values[0, i]


In [1380]:
final_df

,property.type,property.age,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance
0,0.0,0.92,0.60,0.592593,0.638838,0.014568,0.139507
1,0.0,0.08,0.36,0.333333,0.290217,0.033597,2.017231
2,0.0,0.00,0.44,0.407407,0.270328,0.032193,3.604147
3,0.0,0.32,0.40,0.370370,0.360473,0.021280,4.466931
4,0.0,0.76,0.64,0.407407,0.473408,0.020506,5.502485
...,...,...,...,...,...,...,...
820,0.0,0.52,0.40,0.370370,0.577807,0.023473,3592.006252
821,0.0,0.52,0.28,0.259259,0.310571,0.013199,3592.033399
822,0.0,0.68,0.40,0.370370,1.000000,0.008181,3592.088219
823,0.0,0.32,0.80,0.740741,0.588313,0.007832,3594.522272


In [1381]:
sample_scaled

property.type                                      0.0
property.age                                     -0.04
epc.numberHeatedRooms                             0.52
epc.numberHabitableRooms                      0.444444
property.estimatedCurrentValue.floorArea      0.541142
property.estimatedCurrentValue.pricePerSqm     0.01635
Physical Distance                                  0.0
Name: 24445, dtype: object

In [1382]:
from sklearn.neighbors import NearestNeighbors
# Ensure both dataframes have the same columns
X_subject = sample_scaled.values
X_comparables = final_df

# Initialize the KNN model with Euclidean distance
knn_euclidean = NearestNeighbors(n_neighbors=10, metric='euclidean')

# Fit the model on the potential comparables data
knn_euclidean.fit(X_comparables)

# Find the nearest 10 neighbors
distances_euclidean, indices_euclidean = knn_euclidean.kneighbors(sample_scaled.to_frame().T)



In [1383]:
# Retrieve the nearest 10 Potential_Comparable_IDs using the indices
nearest_properties_df = df.iloc[indices_euclidean[0]]

# Get the corresponding distances for the nearest properties
nearest_distances = distances_euclidean[0]

# Add the distances as a new column to the dataframe
nearest_properties_df['KNN_Distance'] = nearest_distances

# Sort the rows by the distance column, from closest to furth~est
nearest_properties_df_sorted = nearest_properties_df.sort_values(by='KNN_Distance')

# Display the sorted rows
print("Rows from potential comparables matching the nearest 10 Property IDs, sorted by distance:")
nearest_properties_df_sorted

Rows from potential comparables matching the nearest 10 Property IDs, sorted by distance:


C:\Users\ArjunShankar\AppData\Local\Temp\ipykernel_18556\1766817513.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nearest_properties_df['KNN_Distance'] = nearest_distances


,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area,KNN_Distance
0,5e431fc077ca272c2eb8b0f1d601b606,rowley ridge 20,NaN,rowley green road,en5 3hj,EN5 3HJ ROWLEY RIDGE 20,0.0,0.0,23.0,51.647251,-0.236892,16.0,17.0,1013.95,5920.150000,0.139507,en5 3hj rowley ridge 20,6000000.0,5917.451551,0.989423
1,f3a87fa3ad3efb49a5d1b418875a5325,27,NaN,totteridge common,n20 8lr,N20 8LR 27,0.0,0.0,2.0,51.632384,-0.218331,10.0,10.0,483.00,10656.260000,2.017231,n20 8lr 27,6250000.0,12939.958592,2.045686
2,588f3470ac40fc72f6616ff96f7bab70,109,NaN,camlet way,en4 0nl,EN4 0NL 109,0.0,0.0,0.0,51.663589,-0.189668,12.0,12.0,452.71,10306.817259,3.604147,en4 0nl 109,4250000.0,9387.908374,3.615638
3,84f2bfa2526ea82af87fe62933a27cbc,28,NaN,beech hill,en4 0jp,EN4 0JP 28,0.0,0.0,8.0,51.663502,-0.175593,11.0,11.0,590.00,7590.771186,4.466931,en4 0jp 28,4650000.0,7881.355932,4.487273
4,d2a6aeebe0242aa61f94608d34c3da7e,8b,NaN,beech hill,en4 0jp,EN4 0JP 8B,0.0,0.0,19.0,51.667090,-0.161736,17.0,12.0,762.00,7398.248031,5.502485,en4 0jp 8b,5900000.0,7742.782152,5.562169
5,325fe66fb1500f9dfb69c8185de4ff61,heath cottage,NaN,marshalls heath lane,al4 8hs,AL4 8HS HEATH COTTAGE,0.0,0.0,1.0,51.824741,51.824741,9.0,9.0,279.00,11513.655914,3507.001853,al4 8hs heath cottage,3025000.0,10842.293907,3507.001884
6,354e9f36b46d0acb14ac4bfe6fde7203,47,NaN,new road,al6 0aq,AL6 0AQ 47,0.0,0.0,3.0,51.825490,51.825490,12.0,12.0,335.00,6268.486567,3507.020556,al6 0aq 47,2075000.0,6194.029851,3507.020579
7,57530bd759373f9c64d178284a579ff2,rundle house,NaN,high street,cm22 7he,CM22 7HE RUNDLE HOUSE,0.0,0.0,25.0,51.825878,51.825878,12.0,12.0,332.00,6401.719880,3507.030290,cm22 7he rundle house,2100000.0,6325.301205,3507.030463
8,c89df6c6af5876cc195a76d7e00d6a91,thatched house,NaN,NaN,ox20 1le,OX20 1LE THATCHED HOUSE,0.0,0.0,0.0,51.826093,51.826093,11.0,11.0,415.00,6910.527711,3507.035634,ox20 1le thatched house,2250000.0,5421.686747,3507.035650
9,f7d278c932356ba5192323e71c3c95a5,turtle farm,NaN,south end,sg10 6ep,SG10 6EP TURTLE FARM,0.0,0.0,25.0,51.826993,51.826993,12.0,12.0,410.00,6242.214634,3507.058157,sg10 6ep turtle farm,2050000.0,5000.000000,3507.058325


In [899]:
nearest_properties_df_sorted=nearest_properties_df_sorted.drop(0)

In [1129]:
new_row = pd.Series({col: np.nan for col in nearest_properties_df_sorted.columns})

# Set the specific column value
new_row["property.estimatedCurrentValue.pricePerSqm"] = 41915.86  # Replace with your actual value

# Append the new row to the DataFrame
nearest_properties_df_sorted = pd.concat(
    [nearest_properties_df_sorted, pd.DataFrame([new_row])],
    ignore_index=True
)
nearest_properties_df_sorted

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area,KNN_Distance
0,a76753651300f89722193df9c51c6e8f,55,apartment 306,bryanston street,w1h 7bz,W1H 7BZ 55 APARTMENT 306,3.0,1.0,25.0,51.513790,-0.163005,7.0,3.0,207.17,37654.670000,0.813961,w1h 7bz 55 apartment 306,7749000.0,37404.064295,1.238227
1,d640d16166fffd52c777dcea399f83fd,19,flat 10,bolsover street,w1w 5na,W1W 5NA 19 FLAT 10,1.0,1.0,24.0,51.521102,-0.145131,7.0,3.0,186.00,24327.096774,1.043262,w1w 5na 19 flat 10,4465000.0,24005.376344,2.426725
2,ba46f60e6ad0729dd6236c87c0cf5f93,chiltern place 66,flat 35,chiltern street,w1u 4ej,W1U 4EJ CHILTERN PLACE 66 FLAT 35,1.0,1.0,23.0,51.520398,-0.177339,7.0,3.0,190.00,50428.989474,1.186680,w1u 4ej chiltern place 66 flat 35,9750000.0,51315.789474,2.481726
3,56dae6a8e44af2ea6b135e888bca2ae6,24,flat 303,paddington street,w1u 5qy,W1U 5QY 24 FLAT 303,0.0,1.0,22.0,51.520900,-0.158293,5.0,3.0,117.00,37153.282051,0.132400,w1u 5qy 24 flat 303,4265000.0,36452.991453,3.115035
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41915.860000,NaN,NaN,NaN,NaN,NaN


In [1135]:
sample['property.estimatedCurrentValue.floorArea']=319.92

In [1133]:
nearest_properties_df_sorted

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area,KNN_Distance
0,a76753651300f89722193df9c51c6e8f,55,apartment 306,bryanston street,w1h 7bz,W1H 7BZ 55 APARTMENT 306,3.0,1.0,25.0,51.513790,-0.163005,7.0,3.0,207.17,37654.670000,0.813961,w1h 7bz 55 apartment 306,7749000.0,37404.064295,1.238227
1,d640d16166fffd52c777dcea399f83fd,19,flat 10,bolsover street,w1w 5na,W1W 5NA 19 FLAT 10,1.0,1.0,24.0,51.521102,-0.145131,7.0,3.0,186.00,24327.096774,1.043262,w1w 5na 19 flat 10,4465000.0,24005.376344,2.426725
2,ba46f60e6ad0729dd6236c87c0cf5f93,chiltern place 66,flat 35,chiltern street,w1u 4ej,W1U 4EJ CHILTERN PLACE 66 FLAT 35,1.0,1.0,23.0,51.520398,-0.177339,7.0,3.0,190.00,50428.989474,1.186680,w1u 4ej chiltern place 66 flat 35,9750000.0,51315.789474,2.481726
3,56dae6a8e44af2ea6b135e888bca2ae6,24,flat 303,paddington street,w1u 5qy,W1U 5QY 24 FLAT 303,0.0,1.0,22.0,51.520900,-0.158293,5.0,3.0,117.00,37153.282051,0.132400,w1u 5qy 24 flat 303,4265000.0,36452.991453,3.115035
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41915.860000,NaN,NaN,NaN,NaN,NaN


In [1242]:
last_valuation=7250000
lower_bound=last_valuation*0.5
print(lower_bound)
upper_bound=last_valuation*1.5
print(upper_bound)

nearest_properties_df_sorted=nearest_properties_df_sorted[
     (nearest_properties_df_sorted['Price'] >= lower_bound) &
     (nearest_properties_df_sorted['Price'] <= upper_bound)
 ]
len(nearest_properties_df_sorted)

3625000.0
10875000.0


5

In [1384]:
nearest_properties_df_sorted = nearest_properties_df_sorted.sort_values(by='KNN_Distance').head(5)
nearest_properties_df_sorted

,Matched_PID,Input_PAON,Input_SAON,Input_Street,Input_Postcode,Property_Index,property.type,property.tenure,property.age,position.lat,position.long,epc.numberHeatedRooms,epc.numberHabitableRooms,property.estimatedCurrentValue.floorArea,property.estimatedCurrentValue.pricePerSqm,Physical Distance,Cleaned_Property_Index,Price,Sold Price Per Area,KNN_Distance
0,5e431fc077ca272c2eb8b0f1d601b606,rowley ridge 20,NaN,rowley green road,en5 3hj,EN5 3HJ ROWLEY RIDGE 20,0.0,0.0,23.0,51.647251,-0.236892,16.0,17.0,1013.95,5920.150000,0.139507,en5 3hj rowley ridge 20,6000000.0,5917.451551,0.989423
1,f3a87fa3ad3efb49a5d1b418875a5325,27,NaN,totteridge common,n20 8lr,N20 8LR 27,0.0,0.0,2.0,51.632384,-0.218331,10.0,10.0,483.00,10656.260000,2.017231,n20 8lr 27,6250000.0,12939.958592,2.045686
2,588f3470ac40fc72f6616ff96f7bab70,109,NaN,camlet way,en4 0nl,EN4 0NL 109,0.0,0.0,0.0,51.663589,-0.189668,12.0,12.0,452.71,10306.817259,3.604147,en4 0nl 109,4250000.0,9387.908374,3.615638
3,84f2bfa2526ea82af87fe62933a27cbc,28,NaN,beech hill,en4 0jp,EN4 0JP 28,0.0,0.0,8.0,51.663502,-0.175593,11.0,11.0,590.00,7590.771186,4.466931,en4 0jp 28,4650000.0,7881.355932,4.487273
4,d2a6aeebe0242aa61f94608d34c3da7e,8b,NaN,beech hill,en4 0jp,EN4 0JP 8B,0.0,0.0,19.0,51.667090,-0.161736,17.0,12.0,762.00,7398.248031,5.502485,en4 0jp 8b,5900000.0,7742.782152,5.562169


In [ ]:
sample

Matched_PID                                   fc803044b7991207fa52b0aac82b3b76
Input_PAON                                                       wyndham house
Input_SAON                                                              flat 2
Input_Street                                                     sloane square
Input_Postcode                                                        sw1w 8ar
Property_Index                                   SW1W 8AR WYNDHAM HOUSE FLAT 2
property.type                                                              3.0
property.tenure                                                            1.0
property.age                                                              20.0
position.lat                                                         51.492019
position.long                                                        -0.157069
epc.numberHeatedRooms                                                      5.0
epc.numberHabitableRooms                            

In [1115]:
nearest_properties_df_sorted['property.estimatedCurrentValue.pricePerSqm'].mean()

37391.00957478995

In [1386]:
nearest_properties_df_sorted['Sold Price Per Area'].mean()*sample['property.estimatedCurrentValue.floorArea']

7590819.814661502

In [1387]:
nearest_properties_df_sorted['property.estimatedCurrentValue.pricePerSqm'].mean()*sample['property.estimatedCurrentValue.floorArea']

7245238.552377117

In [ ]:
# # Step 1: Sort by physical distance
# nearest_5 = nearest_properties_df_sorted.sort_values(by='KNN_Distance').head(10).copy()

# # Step 2: Avoid division by zero (if distance = 0, add a tiny number)
# nearest_5['Weight'] = 1 / (nearest_5['KNN_Distance'] + 1e-6)

# # Step 3: Calculate weighted average price
# weighted_mean_price = (nearest_5['Price'] * nearest_5['Weight']).sum() / nearest_5['Weight'].sum()

# # Step 4: Define 50% price bounds
# lower_bound = weighted_mean_price * 0.5
# upper_bound = weighted_mean_price * 1.5

# # Step 5: Filter original dataframe based on those bounds
# final_comparables = nearest_properties_df_sorted[
#     (nearest_properties_df_sorted['Price'] >= lower_bound) &
#     (nearest_properties_df_sorted['Price'] <= upper_bound)
# ]

# # Optional: print weighted mean and bounds
# print(f"Weighted Mean Price: {weighted_mean_price:.2f}")
# print(f"Allowed Price Range: {lower_bound:.2f} - {upper_bound:.2f}")

Weighted Mean Price: 4207949.75
Allowed Price Range: 2103974.87 - 6311924.62


In [503]:
subject_price=5500000

In [512]:
comparables_mean_price=final_comparables['Price'].mean()     
comparables_mean_price

np.float64(3060714.285714286)

In [505]:
import numpy as np

accuracy = (1 - abs(subject_price - comparables_mean_price) / subject_price) * 100

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 69.59%


In [506]:
new_df1 = pd.read_excel("Data/premium_property_enrich_individual_properties.xlsx")

In [510]:
len(new_df1[new_df1['Price']>5000000])


3119